# Experiment Notebook
Notebook to inspect the results of a single experiment

In [ ]:
import pandas as pd
import graphviz
import typing
import pprint
import matplotlib.pyplot as plt

from collections import defaultdict


from lib.metrics import *
from lib.message import *
from lib.analysis import *
from lib.experiment import PubSubExperiment, PubSubExperimentResults

EXPERIMENT = "experiments/plumtree-quick.json"
# EXPERIMENT = "experiments/kadpubsub-quick.json"


In [ ]:
experiment_results = PubSubExperimentResults.load_from_file(EXPERIMENT)
exp = (
    KadPubSubAnalyzer.from_experiment_results(experiment_results)
    if "kad" in EXPERIMENT
    else PubSubAnalyzer.from_experiment_results(experiment_results)
)
pprint.pprint(exp.experiment)


In [ ]:
### Preconditions

# First message was sent after the last subscription
from lib.experiment import PROTOCOL_KADPUBSUB


def check_first_message_after_last_subscription():
    metrics = exp.metrics(ty=(PubSubMessageSent, PubSubSubscribe))
    last_subscribe = {}
    first_send = {}
    for metric in metrics:
        if isinstance(metric, PubSubSubscribe):
            last_subscribe[metric.topic] = metric.timestamp
        elif isinstance(metric, PubSubMessageSent):
            if metric.topic not in first_send:
                first_send[metric.topic] = metric.timestamp

    for topic, timestamp in first_send.items():
        if topic not in last_subscribe:
            print(f"Topic {topic} was never subscribed to")
            continue
        if timestamp < last_subscribe[topic]:
            raise Exception(f"First message sent to {topic} before last subscription")


def check_dirty_topic_routing_tables():
    if exp.experiment.protocol != PROTOCOL_KADPUBSUB:
        return

    for snapshot in exp.metrics(ty=RoutingTableSnapshot):
        if snapshot.topic == "":
            continue
        subscribers = exp.topic_subscribers(snapshot.topic)
        for node in [n for bucket in snapshot.buckets for n in bucket]:
            if node not in subscribers:
                raise Exception("You fucked up the routing tables, good job")


check_first_message_after_last_subscription()
check_dirty_topic_routing_tables()


In [ ]:
exp.reliability().resample("20ms").mean().fillna(method='bfill').plot(title="Reliability")


In [ ]:
exp.reliability().plot(kind="hist", logy=True)


In [ ]:
pd.DataFrame(
    {
        "Message Sends": pd.Series(
            1, index=list(map(lambda m: m.timestamp, exp.metrics(ty=PubSubMessageSent)))
        ),
        "Message Receives": pd.Series(
            1,
            index=list(
                map(lambda m: m.timestamp, exp.metrics(ty=PubSubMessageReceived))
            ),
        )
        .groupby(level=0)
        .sum(),
        "Node Boot Times": pd.Series(
            1, index=list(map(lambda m: m.timestamp, exp.metrics(ty=Boot)))
        ),
        "Node Shutdown Times": pd.Series(
            1, index=list(map(lambda m: m.timestamp, exp.metrics(ty=Shutdown)))
        ),
        "Node Subscriptions": pd.Series(
            1, index=list(map(lambda m: m.timestamp, exp.metrics(ty=PubSubSubscribe)))
        ),
    }
).resample("1s").sum().plot(logy=True)


In [ ]:
exp.publish_latency().plot(kind="hist", title="Publish Latencies")


In [ ]:
print(f"Redundancy: {exp.redundancy()}")


In [ ]:
print(f"Network usefullness fraction: {exp.network_usage_usefullness_fraction()}")

In [ ]:
for snapshot in exp.metrics(ty=RoutingTableSnapshot):
    if snapshot.topic == "":
        continue
    subscribers = exp.topic_subscribers(snapshot.topic)
    for node in [n for bucket in snapshot.buckets for n in bucket]:
        if node not in subscribers:
            raise Exception("You fucked up the routing tables, good job")


In [ ]:
# Graph of message with reliability 1
graph = None
try:
    reliability_1 = [k for k, v in exp.messages_reliability().items() if v == 1.0][0]
    graph = exp.message_graph(reliability_1)
except:
    print("No message with reliability 1")
graph


In [ ]:
# Graph of message with reliability < 1
graph = None
message_id = None
try:
    message_id = sorted(
        [(k, v) for k, v in exp.messages_reliability().items() if v < 1],
        key=lambda t: t[1],
    )[0][0]
    print(
        f"Message {message_id} has reliability {exp.messages_reliability()[message_id]}"
    )
    graph = exp.message_graph(message_id)
except IndexError:
    print("No message with reliability < 1")
graph


In [ ]:
delivers = exp.metrics(ty=PubSubMessageReceived, filter=lambda m: m.message_id == message_id and m.delivered)
delivers

In [ ]:
# Ignora isto
# message = message_id
# message_timestamp = exp.timestamp_of_message(message)
# message_send = exp.metrics(
#     ty=PubSubMessageSent, filter=lambda m: m.message_id == message
# )[0]
# message_receives = exp.metrics(
#     ty=PubSubMessageReceived, filter=lambda m: m.message_id == message
# )
# topic = exp.topic_of_message(message)
# subscribers = exp.topic_subscribers(topic)
# assert all(map(lambda m: m.node in subscribers, message_receives))
# snapshots = exp.routing_table_snapshot(subscribers, topic, message_timestamp)
# missing_nodes = subscribers - set(map(lambda m: m.node, message_receives))